In [ ]:
#STEP 1

# Imports
from materials import materials
import pandas as pd
import os
from pyXSteam.XSteam import XSteam
import numpy as np

# INPUTS VARIABLES

# Always input these variables
inside_diameter = 9 #Enter the desired inside diameter as a decimal

# Optional variables
standard_only = False #False will include entries without identification or schedule
number_of_entries = 2 #Enter an integer if you only want to see a certain number of results, default is to see all entries
#TODO update your function above to work with the materials.py and make this block work with section viii= True
## Filter by Codes based on pressure and temperature if desired
section_viii = False #Set to True to only return results that pass Section VIII Div 1. UG-27 for design pressure and temperature
design_pressure = 5 #psig, must be entered if code filter is set to True
design_temperature = 8 #F, must be entered if code filter is set to True
material = None #'304', '316', '106 Gr B' are the currently available materials, must be entered if code filter is set to True
seamless = False #Set to False to apply a 0.85 strength reduction factor for welded pipe

# Fluid Variables
# Pressure (psig), Temperature (°F), Mass Flow Rate (lbm/s), Target Velocity (ft/s)
fluid_pressure = 15  # psig
fluid_temperature = 100  # °F
mass_flow_rate = 10  # lbm/s
target_velocity = 15  # ft/s

    

In [228]:
#STEP 2 
#READ THE CSV AND CALCULATE INSIDE DIAMETER AND ADDS LAST COLUMN 

df = pd.read_csv(r'pipe\pipe_sizes.csv')
df['Inside Diameter'] = df['Outside Diameter'] - 2 * df['Wall Thickness']


In [229]:
 #STEP 3 
# takes in df and inside_diameter, returns a DataFrame of pipe sizes closest to the inside diameter
def search_by_inside_diameter(df,
                              inside_diameter,
                              standard_only=True,
                              number_of_entries=None,
                              b31_1=False,
                              b31_3=False,
                              section_vii=False,
                              design_pressure=0,
                              design_temperature=0,
                              material=None,
                              seamless=True):
    """
    Searches for pipe entries closest to the specified inside diameter,
    with optional code compliance using materials.py.
    """
    if inside_diameter is None:
        raise ValueError("You must provide a target inside diameter.")

    filtered_df = df.copy()

    # Filter standard pipes only 
    if standard_only:
        filtered_df['Standard'] = ~(filtered_df['Identification'].isna() & filtered_df['Schedule'].isna())
        filtered_df = filtered_df[filtered_df['Standard']].drop(columns='Standard')

    # Filter by design codes if requested
    if any([b31_1, b31_3, section_vii]):
        if not material or design_pressure == 0 or design_temperature == 0:
            raise ValueError("Material, design_pressure, and design_temperature are required for code filtering.")

        # Use check_code_compliance from cell 6
        filtered_df = filtered_df[filtered_df.apply(
            lambda row: check_code_compliance(row, material, design_pressure, design_temperature,
                                              b31_1, b31_3, section_vii, seamless), axis=1)]

    # Compute absolute difference from target inside diameter
    filtered_df['ID Diff'] = abs(filtered_df['Inside Diameter'] - inside_diameter)

    # Compute the flow area in in^2
    filtered_df['Area'] = (filtered_df['Inside Diameter'] ** 2) * 3.14159 / 4

    # Sort by how close the inside diameter is
    filtered_df = filtered_df.sort_values(by='ID Diff')

    # Limit number of results if specified
    if number_of_entries:
        filtered_df = filtered_df.head(number_of_entries)

    # Drop helper column and reset index
    filtered_df = filtered_df.drop(columns=['ID Diff'])

    return filtered_df.reset_index(drop=True)

filtered_df = search_by_inside_diameter(df, inside_diameter)
filtered_df

,NPS,Identification,Schedule,Outside Diameter,Wall Thickness,Weight,Inside Diameter,Area
0,10.0,NaN,120,10.75,0.844,89.38,9.062,64.496720
1,10.0,XXS,140,10.75,1.000,104.23,8.750,60.131996
2,10.0,NaN,100,10.75,0.719,77.10,9.312,68.104444
3,10.0,NaN,160,10.75,1.125,115.75,8.500,56.744969
4,10.0,NaN,80,10.75,0.594,64.49,9.562,71.810342
...,...,...,...,...,...,...,...,...
333,44.0,STD,NaN,44.00,0.375,174.88,43.250,1469.135111
334,46.0,XS,NaN,46.00,0.500,243.20,45.000,1590.429938
335,46.0,STD,NaN,46.00,0.375,182.90,45.250,1608.150469
336,48.0,XS,NaN,48.00,0.500,253.89,47.000,1734.943077


In [230]:
#step 4
# functions that lookup water density, check code compliance, and calculate velocity

# Simplified material stress lookup table
MATERIALS_DATA = {
    '106 Gr B': {'max_stress': 20000},
    '304': {'max_stress': 18000},
    '316': {'max_stress': 17000},
}

def lookup_water_density(pressure, temperature):
    """
    Returns water density in lbm/ft³ based on pressure and temperature.
    Uses XSteam for accurate property lookup.
    """
    steam_table_imp = XSteam(XSteam.UNIT_SYSTEM_FLS)  # imperial units (psig, F)
    return steam_table_imp.rho_pt(pressure, temperature)

def check_code_compliance(row, material, pressure, temperature, b31_1, b31_3, section_vii, seamless):
    """
    Simplified compliance check using Barlow’s formula:
    P = (2 * S * t) / D
    """
    if material not in MATERIALS_DATA:
        return False

    S = MATERIALS_DATA[material]['max_stress']
    if not seamless:
        S *= 0.85

    D = row['Outside Diameter']
    t = row['Wall Thickness']
    
    P_allow = (2 * S * t) / D
    return pressure <= P_allow



def calculate_velocity(mass_flow_rate, density, area_in2):
    """
    Calculate fluid velocity in ft/s given:
    - mass_flow_rate in lbm/s
    - density in lbm/ft³
    - area in in²
    """
    volumetric_flow_ft3s = mass_flow_rate / density
    area_ft2 = area_in2 / 144
    return volumetric_flow_ft3s / area_ft2

def search_by_fluid_velocity(df,
                             fluid_pressure,
                             fluid_temperature,
                             mass_flow_rate,
                             target_velocity,
                             standard_only=True,
                             number_of_entries=None,
                             b31_1=False,
                             b31_3=False,
                             section_vii=False,
                             design_pressure=0,
                             design_temperature=0,
                             material=None,
                             seamless=True):
    # Validate required inputs
    if None in [fluid_pressure, fluid_temperature, mass_flow_rate, target_velocity]:
        raise ValueError("fluid_pressure, fluid_temperature, mass_flow_rate, and target_velocity must be specified.")

    # Step 1: Estimate fluid density
    density = lookup_water_density(fluid_pressure, fluid_temperature)

    # Step 2: Calculate required pipe flow area
    volumetric_flow = mass_flow_rate / density  # ft³/s
    required_area_ft2 = volumetric_flow / target_velocity
    required_area_in2 = required_area_ft2 * 144  # convert to in²

    # Step 3: Filter and rank based on closest area
    working_df = df.copy()
    working_df = working_df.assign(
        AreaDiff=np.abs(working_df['Area'] - required_area_in2)
    ).sort_values('AreaDiff')

    # Step 4: Apply standard-only filter
    if standard_only:
        working_df = working_df[
            working_df['Identification'].isnull() |
            (working_df['Identification'].str.upper() == 'STD')
        ]

    # Step 5: Code compliance filtering
    if any([b31_1, b31_3, section_vii]):
        if design_pressure == 0 or design_temperature == 0 or material is None:
            raise ValueError("Code filtering requires design_pressure, design_temperature, and material.")
        working_df = working_df[working_df.apply(
            lambda row: check_code_compliance(row, material, design_pressure, design_temperature,
                                              b31_1, b31_3, section_vii, seamless), axis=1)]

    # Step 6: Calculate velocity for each row
    working_df['Velocity (ft/s)'] = calculate_velocity(mass_flow_rate, density, filtered_df["Area"])
    
    # Step 7: Filter results to show only those pipes that give a fluid velocity less than the target velocity
    working_df = working_df[working_df['Velocity (ft/s)'] < target_velocity]

    
    # Step 8: Limit results
    if number_of_entries is not None:
        working_df = working_df.head(number_of_entries)

    return working_df[[
        'NPS', 'Schedule', 'Identification', 'Inside Diameter',
        'Outside Diameter', 'Wall Thickness', 'Area', 'Velocity (ft/s)'
    ]]


density = lookup_water_density(fluid_pressure, fluid_temperature)
velocity_fts = calculate_velocity(mass_flow_rate, density, filtered_df["Area"])
velocity_fts
fluid_velocity = search_by_fluid_velocity(filtered_df, fluid_pressure, fluid_temperature, mass_flow_rate, target_velocity)
fluid_velocity

,NPS,Schedule,Identification,Inside Diameter,Outside Diameter,Wall Thickness,Area,Velocity (ft/s)
176,1.25,30,NaN,1.426,1.66,0.117,1.597087,14.500701
174,1.25,10,NaN,1.442,1.66,0.109,1.633127,14.180696
175,1.25,10S,NaN,1.442,1.66,0.109,1.633127,14.180696
167,1.25,5S,NaN,1.530,1.66,0.065,1.838537,12.596364
168,1.25,5,NaN,1.530,1.66,0.065,1.838537,12.596364
...,...,...,...,...,...,...,...,...
329,40.00,NaN,STD,39.250,40.00,0.375,1209.953936,0.019140
331,42.00,NaN,STD,41.250,42.00,0.375,1336.402934,0.017329
333,44.00,NaN,STD,43.250,44.00,0.375,1469.135111,0.015764
335,46.00,NaN,STD,45.250,46.00,0.375,1608.150469,0.014401


In [231]:
# step 5 
# it returns the matching pipe info based on a specific NPS, schedule or identification.
import pandas as pd
import numpy as np

def calculate_velocity(mass_flow_rate, density, area_in2):
    """Calculate fluid velocity in ft/s."""
    volumetric_flow = mass_flow_rate / density  # ft³/s
    area_ft2 = area_in2 / 144  # in² → ft²
    return volumetric_flow / area_ft2

def get_pipe_info(df, nps, schedule=None, identification=None,
                  fluid_pressure=None, fluid_temperature=None,
                  mass_flow_rate=None):
    """
    Returns details for a specific pipe size and optional fluid flow info as a DataFrame row.
    """
    if nps is None or (schedule is None and identification is None):
        raise ValueError("Must specify both NPS and either Schedule or Identification.")

    # Lookup matching pipe
    if schedule is not None:
        row = df[(df['NPS'] == nps) & (df['Schedule'].astype(str) == str(schedule))]
    else:
        row = df[(df['NPS'] == nps) &
                 (df['Identification'].astype(str).str.upper() == str(identification).upper())]

    if row.empty:
        raise ValueError("No matching pipe found in database.")

    row = row.iloc[0]

    # Extract geometry
    od = row['Outside Diameter']
    wt = row['Wall Thickness']
    id_ = row['Inside Diameter']
    area = row['Area']

    # Base output dictionary
    result = {
        'NPS': nps,
        'Schedule': schedule,
        'Identification': identification,
        'Outside Diameter (in)': od,
        'Wall Thickness (in)': wt,
        'Inside Diameter (in)': id_,
        'Flow Area (in²)': area,
    }

    # If fluid details given, compute density and velocity
    if None not in [fluid_pressure, fluid_temperature, mass_flow_rate]:
        density = lookup_water_density(fluid_pressure, fluid_temperature)  # you’ll need your function here
        velocity = calculate_velocity(mass_flow_rate, density, area)
        result.update({
            'Water Density (lbm/ft³)': density,
            'Mass Flow Rate (lbm/s)': mass_flow_rate,
            'Fluid Velocity (ft/s)': velocity
        })

    # Return as a DataFrame for table display
    return pd.DataFrame([result])

# Example call:
pipe_info_df = get_pipe_info(
    filtered_df,
    nps=6,
    schedule=40,
    fluid_pressure=150,      # example values
    fluid_temperature=200,   # example values
    mass_flow_rate=5.0       # lbm/s
)



get_pipe_info(filtered_df, nps=6, schedule=40, identification=None,
              fluid_pressure=fluid_pressure, fluid_temperature=fluid_temperature,
              mass_flow_rate=mass_flow_rate)

# functions of code 
# step 3 searches by inside diameter and returns a DataFrame of pipe sizes closest to the target inside diameter. search_by_inside_diameter
# step 4  functions that lookup water density, check code compliance, and calculate velocity and outputs a dataframe of pipe sizes that give the closest to the target fluiod velocity search_by_fluid_velocity
#step 5 returns the matching pipe info based on a specific NPS, schedule or identification(it returns density, mass flow rate, and velocity if fluid pressure, temperature, and mass flow rate are provided).

,NPS,Schedule,Identification,Outside Diameter (in),Wall Thickness (in),Inside Diameter (in),Flow Area (in²),Water Density (lbm/ft³),Mass Flow Rate (lbm/s),Fluid Velocity (ft/s)
0,6,40,None,6.625,0.28,6.065,28.890238,62.17917,10,0.801616
